In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import matplotlib
import numpy as np

for font in font_manager.findSystemFonts('/Users/leonardokuffo/Library/Fonts/'):
    font_manager.fontManager.addfont(font)

matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')
matplotlib.rc('font', family='Droid Serif') 

In [2]:
import seaborn as sns
from matplotlib.colors import LogNorm, Normalize
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import LogFormatter, FormatStrFormatter
from matplotlib.ticker import FixedLocator

from matplotlib.patches import Rectangle

arch_mapping = {
    'ZEN4_INSITU': 'ZEN 4\nx86-64 | AVX512',
    'ZEN4_AWS': 'ZEN 4\nx86-64 | AVX512',
    'SAPPHIRE': 'INTEL SAPPHIRE RAPIDS\nx86-64 | AVX512',
    'ZEN3': 'ZEN 3\nx86-64 | AVX2',
    'GRAVITON4': 'GRAVITON 4\nARM | NEON',
    'M1': 'M1\nARM | NEON '
}

font = {'size': 8}
matplotlib.rc('font', **font)

In [3]:
results = {
    'arch': [],
    'L2 =8': [],
    'L2 (8,32)': [],
    'L2 >32': [],
    'L2 all': [],
    'IP =8': [],
    'IP (8,32)': [],
    'IP >32': [],
    'IP all': [],
    'L1 =8': [],
    'L1 (8,32)': [],
    'L1 >32': [],
    'L1 all': [],
    'L2 >4k': [],
    'L1 >4k': [],
    'IP >4k': [],
}
for i, arch in enumerate(['SAPPHIRE', 'ZEN4_AWS', 'ZEN3', 'GRAVITON4']):
    results['arch'].append(arch)
    for j, alg in enumerate(['L2', 'IP', 'L1']):
    
        pdx = pd.read_csv(f'../{arch}/PURESCAN_PDX_{alg}.csv', index_col=False)
        pdx = pdx[['dataset', 'avg']]
        
        hor = pd.read_csv(f'../{arch}/PURESCAN_SIMD_{alg}.csv', index_col=False)
        hor = hor[['dataset', 'avg']]
        
        df = pdx.merge(
            hor, on=['dataset'], how='inner', suffixes=('_pdx', '_simd')
        )
        df['speedup'] = df['avg_simd'] / df['avg_pdx']
        df['n_vectors'] = df['dataset'].apply(lambda x: int(x.split('x')[0]))
        df['dimensions'] = df['dataset'].apply(lambda x: int(x.split('x')[1]))
        
        xlabels = [8, 16, 32, 64, 128, 256, 512, 768, 1024, 1536, 2048, 4096, 8192]
        ylabels = [64, 128, 512, 1024, 4096, 8192, 16384, 65536, 131072] 

        df = df[
            (df['n_vectors'].isin(ylabels)) & 
            (df['dimensions'].isin(xlabels))
        ]

        speedup_low = df[(df['dimensions'] <= 32) & (df['dimensions'] != 8)]['speedup'].mean()
        speedup_8 = df[df['dimensions'] == 8]['speedup'].mean()
        speedup_fair = df[df['dimensions'] > 32]['speedup'].mean()
        speedup_high = df[df['dimensions'] >= 4096]['speedup'].max()
        speedup_all = df['speedup'].mean()

        # print('Average speedup D = 8 dims', df[df['dimensions'] == 8]['speedup'].mean())
        # print('Average speedup 8 < D <= 32 dims', df[(df['dimensions'] <= 32) & (df['dimensions'] != 8)]['speedup'].mean())
        # print('Average speedup D > 32 dims', df[df['dimensions'] > 32]['speedup'].mean())
        # print('Average speedup all', df['speedup'].mean())

        results[str(alg) + ' =8'].append(speedup_8)
        results[str(alg) + ' (8,32)'].append(speedup_low)
        results[str(alg) + ' >32'].append(speedup_fair)
        results[str(alg) + ' all'].append(speedup_all)
        results[str(alg) + ' >4k'].append(speedup_high)
df_results = pd.DataFrame(results)
df_results.set_index('arch', inplace=True)
df_results.loc['Average'] = df_results.mean(numeric_only=True)
df_results = df_results.round(decimals=1)
df_results

/var/folders/g_/tfhsysh13vz_74rpmgswx9hc0000gn/T/ipykernel_18361/2667390891.py:23: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  pdx = pd.read_csv(f'../{arch}/PURESCAN_PDX_{alg}.csv', index_col=False)
/var/folders/g_/tfhsysh13vz_74rpmgswx9hc0000gn/T/ipykernel_18361/2667390891.py:26: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  hor = pd.read_csv(f'../{arch}/PURESCAN_SIMD_{alg}.csv', index_col=False)
/var/folders/g_/tfhsysh13vz_74rpmgswx9hc0000gn/T/ipykernel_18361/2667390891.py:26: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  hor = pd.read_csv(f'../{arch}/PURESCAN_SIMD_{alg}.csv', index_col=False)
/var/folders/g_/tfhsysh13vz_74rpmgswx9hc0000gn/T/ipykernel_18361/2667390891.py:26: ParserWarning: Length of header or names does not match length of data. Thi

,L2 =8,"L2 (8,32)",L2 >32,L2 all,IP =8,"IP (8,32)",IP >32,IP all,L1 =8,"L1 (8,32)",L1 >32,L1 all,L2 >4k,L1 >4k,IP >4k
arch,,,,,,,,,,,,,,,
SAPPHIRE,5.8,2.4,1.3,1.8,5.6,2.4,1.2,1.7,5.3,2.5,1.2,1.7,1.6,1.6,1.5
ZEN4_AWS,7.4,2.7,1.4,2.0,6.6,2.5,1.4,2.0,6.7,2.8,1.4,2.0,2.1,2.0,2.1
ZEN3,6.2,3.3,1.7,2.3,5.9,3.1,1.5,2.1,7.4,3.5,1.4,2.2,2.1,1.6,2.1
GRAVITON4,2.7,1.5,1.8,1.8,3.1,1.8,1.9,2.0,2.6,1.5,1.9,1.9,2.6,2.8,3.1
Average,5.5,2.5,1.5,2.0,5.3,2.4,1.5,2.0,5.5,2.6,1.5,2.0,2.1,2.0,2.2


In [4]:
results = {
    'arch': [],
    'L2 =8': [],
    'L2 (8,32)': [],
    'L2 >32': [],
    'L2 all': [],
    'IP =8': [],
    'IP (8,32)': [],
    'IP >32': [],
    'IP all': [],
    'L1 =8': [],
    'L1 (8,32)': [],
    'L1 >32': [],
    'L1 all': [],
    'L2 >4k': [],
    'L1 >4k': [],
    'IP >4k': [],
}
for i, arch in enumerate(['SAPPHIRE', 'ZEN4_AWS', 'ZEN3', 'GRAVITON4']):
    results['arch'].append(arch)
    for j, alg in enumerate(['L2', 'IP', 'L1']):
        print(arch, alg)
        pdx = pd.read_csv(f'../{arch}/PURESCAN_PDX_{alg}.csv', index_col=False)
        pdx = pdx[['dataset', 'avg']]
        
        hor = pd.read_csv(f'../{arch}/PURESCAN_SIMD_{alg}.csv', index_col=False)
        hor = hor[['dataset', 'avg']]
        
        df = pdx.merge(
            hor, on=['dataset'], how='inner', suffixes=('_pdx', '_simd')
        )
        df['speedup'] = df['avg_simd'] / df['avg_pdx']
        df['n_vectors'] = df['dataset'].apply(lambda x: int(x.split('x')[0]))
        df['dimensions'] = df['dataset'].apply(lambda x: int(x.split('x')[1]))
        
        # xlabels = [8, 16, 32, 64, 128, 256, 512, 768, 1024, 1536, 2048, 4096, 8192]
        ylabels = [64, 128, 512, 1024, 4096, 8192, 16384, 65536, 131072] 

        # df = df[
        #     (df['n_vectors'].isin(ylabels)) # & 
        #     # (df['dimensions'].isin(xlabels))
        # ]

        speedup_low = df[(df['dimensions'] <= 32) & (df['dimensions'] != 8)]['speedup'].max()
        speedup_8 = df[df['dimensions'] == 8]['speedup'].max()
        print(df[df['dimensions'] == 8][['dataset', 'speedup']])
        speedup_fair = df[df['dimensions'] > 32]['speedup'].max()
        speedup_high = df[df['dimensions'] >= 4096]['speedup'].max()
        speedup_all = df['speedup'].mean()

        results[str(alg) + ' =8'].append(speedup_8)
        results[str(alg) + ' (8,32)'].append(speedup_low)
        results[str(alg) + ' >32'].append(speedup_fair)
        results[str(alg) + ' all'].append(speedup_all)
        results[str(alg) + ' >4k'].append(speedup_high)
df_results = pd.DataFrame(results)
df_results.set_index('arch', inplace=True)
df_results.loc['Average'] = df_results.mean(numeric_only=True)
df_results = df_results.round(decimals=1)
df_results

SAPPHIRE L2
       dataset   speedup
0         64x8  3.868248
15       128x8  5.527904
30       512x8  8.380674
45      1024x8  8.343450
60      4096x8  6.013184
75      8192x8  6.113005
90     16384x8  6.084526
105    65536x8  5.714267
120   131072x8  2.527779
135   262144x8  2.329502
150  1048576x8  2.611990
SAPPHIRE IP
       dataset   speedup
0         64x8  3.712860
15       128x8  5.347413
30       512x8  7.957625
45      1024x8  8.542240
60      4096x8  5.640296
75      8192x8  5.653416
90     16384x8  5.713799
105    65536x8  5.124582
120   131072x8  2.294127
135   262144x8  2.314834
150  1048576x8  2.641137
SAPPHIRE L1
       dataset   speedup
0         64x8  3.543181
15       128x8  4.957072
30       512x8  7.188078
45      1024x8  7.545352
60      4096x8  5.718240
75      8192x8  5.780265
90     16384x8  5.804527
105    65536x8  5.102897
120   131072x8  2.266638
135   262144x8  2.266914
150  1048576x8  2.437443
ZEN4_AWS L2
       dataset   speedup
0         64x8  4.539446
15

/var/folders/g_/tfhsysh13vz_74rpmgswx9hc0000gn/T/ipykernel_18361/4225423772.py:23: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  pdx = pd.read_csv(f'../{arch}/PURESCAN_PDX_{alg}.csv', index_col=False)
/var/folders/g_/tfhsysh13vz_74rpmgswx9hc0000gn/T/ipykernel_18361/4225423772.py:26: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  hor = pd.read_csv(f'../{arch}/PURESCAN_SIMD_{alg}.csv', index_col=False)
/var/folders/g_/tfhsysh13vz_74rpmgswx9hc0000gn/T/ipykernel_18361/4225423772.py:26: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  hor = pd.read_csv(f'../{arch}/PURESCAN_SIMD_{alg}.csv', index_col=False)
/var/folders/g_/tfhsysh13vz_74rpmgswx9hc0000gn/T/ipykernel_18361/4225423772.py:26: ParserWarning: Length of header or names does not match length of data. Thi

,L2 =8,"L2 (8,32)",L2 >32,L2 all,IP =8,"IP (8,32)",IP >32,IP all,L1 =8,"L1 (8,32)",L1 >32,L1 all,L2 >4k,L1 >4k,IP >4k
arch,,,,,,,,,,,,,,,
SAPPHIRE,8.4,4.4,1.9,1.7,8.5,6.3,2.4,1.6,7.5,4.4,1.8,1.6,1.6,1.6,1.5
ZEN4_AWS,10.0,4.6,2.1,1.8,9.1,4.2,2.6,1.8,7.8,3.9,2.0,1.9,2.1,2.0,2.1
ZEN3,8.7,6.0,3.4,2.0,8.6,4.7,2.9,1.9,9.3,5.4,2.2,2.0,2.1,1.6,2.1
GRAVITON4,3.0,1.8,2.6,1.7,3.5,2.3,3.2,1.8,3.0,1.7,3.0,1.8,2.6,2.8,3.1
Average,7.5,4.2,2.5,1.8,7.4,4.4,2.8,1.8,6.9,3.9,2.3,1.8,2.1,2.0,2.2
